In [2]:
print("jai mata jii")

jai mata jii


In [3]:
import cv2
import os

In [4]:
# data_dir = "./images"  
# if not os.path.exists(data_dir):
#     os.makedirs(data_dir)

# number_of_classes = 12
# dataset_size = 25
 
# # Initialize camera
# cap = cv2.VideoCapture(0)  

# if not cap.isOpened():
#     print("Error: Could not open camera.")
# else:
#     for j in range(number_of_classes):
#         # Create directory for each class if it doesn't exist
#         class_dir = os.path.join(data_dir, str(j))
#         if not os.path.exists(class_dir):
#             os.makedirs(class_dir)

#         print(f'Collecting data for class {j}')

#         # Wait for user to be ready
#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 print("Could not read frame.")
#                 break 

#             cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,cv2.LINE_AA)
#             cv2.imshow('frame', frame)
#             if cv2.waitKey(25) & 0xFF == ord('q'):
#                 break

#         # Capture dataset_size frames for each class
#         counter = 0
#         while counter < dataset_size:
#             ret, frame = cap.read()
#             if not ret:
#                 print("Error: Could not read frame.")
#                 break  

#             # Show and save the frame
#             cv2.imshow('frame', frame)
#             cv2.imwrite(os.path.join(class_dir, f'{counter}.jpg'), frame)
#             counter += 1

#             if cv2.waitKey(25) & 0xFF == ord('q'):
#                 print("Interrupted by user.")
#                 break  # Allow exiting early if 'q' is pressed

#     # Release the camera and close windows after all data is collected
#     cap.release()
#     cv2.destroyAllWindows()


In [5]:
# process the data
# create the data
import mediapipe as mp
import matplotlib.pyplot as plt
import pickle

In [6]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

data_dir= './images'

data = []
labels = []
for dir_ in os.listdir(data_dir):
    for img_path in os.listdir(os.path.join(data_dir, dir_)):
        data_aux = []

        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(data_dir, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            data.append(data_aux)
            labels.append(dir_)

f = open('images.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()

d:\Program Files\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
import numpy as np
# data_num = np.array(data)
# print([len(item) if isinstance(item, list) else type(item) for item in data])
# data_np_list = [np.array(item) for item in data]
# data_np_list
max_length = 84
data_padded = [item + [0] * (max_length - len(item)) if len(item) < max_length else item for item in data]
data_np = np.array(data_padded)
# print(data_np)
print([len(item) for item in data_np])


[array([0.03031078, 0.22211024, 0.09416422, 0.20692763, 0.14641122,
        0.13756236, 0.16849463, 0.06128153, 0.18070813, 0.00618139,
        0.11878339, 0.06595689, 0.12711193, 0.00964195, 0.11707381,
        0.07219937, 0.11165222, 0.09746784, 0.08166333, 0.05517116,
        0.08620613, 0.        , 0.08157509, 0.0778589 , 0.07972025,
        0.098198  , 0.04251163, 0.05498314, 0.04655106, 0.00028542,
        0.04726886, 0.07877347, 0.04803228, 0.10230777, 0.        ,
        0.05990699, 0.00532293, 0.02287811, 0.01199896, 0.07465684,
        0.01365874, 0.09677851, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [ ]:
# for dir_ in os.listdir(data_dir):
#     for img_path in os.listdir(os.path.join(data_dir, dir_))[:1]:
#         img = cv2.imread(os.path.join(data_dir, dir_, img_path))
#         img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#         plt.figure()
#         plt.imshow(img_rgb)
# plt.show()

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [10]:
data_dict = pickle.load(open('./images.pickle', 'rb'))
data = np.array(data_padded)
labels = np.array(data_dict['labels'])

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

model = RandomForestClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))

f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

100.0% of samples were classified correctly !


In [ ]:
# Load the model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# Initialize the camera and mediapipe tools
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Label dictionary for predictions
labels_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'L', 7: 'Y', 8: 'I', 9: 'K', 10: 'I LOVE YOU', 11: 'SAVE'}

while True:
    data_aux = []
    x_ = []
    y_ = []

    # Capture frame-by-frame
    ret, frame = cap.read()
    H, W, _ = frame.shape

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks on the hand
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

        # Collecting landmarks for each detected hand
        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                x_.append(x)
                y_.append(y)

            # Append x and y offsets relative to the minimum x and y values
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        # Ensure data_aux has exactly 84 features by padding or truncating
        if len(data_aux) < 84:
            data_aux.extend([0] * (84 - len(data_aux)))  # Pad with zeros
        elif len(data_aux) > 84:
            data_aux = data_aux[:84]  # Truncate to 84 if needed

        # Define bounding box for visualization
        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10
        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        # Predict using the model
        prediction = model.predict([np.array(data_aux)])
        predicted_character = labels_dict[int(prediction[0])]

        # Display the prediction
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)

    # Show the frame with annotations
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

d:\Program Files\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
d:\Program Files\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
d:\Program Files\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
d:\Program Files\Python312\Lib\site-package